In [1]:
import gymnasium as gym
import torch, torch.nn as nn, torch.optim as optim
import numpy as np
from torch.distributions.categorical import Categorical

# --- small policy ---
class Policy(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, act_dim)
        )
    def forward(self, x): return self.net(x)           # logits
    def dist(self, x): return Categorical(logits=self(x))

def rollout_group(env_fn, policy, G, max_steps=256, gamma=0.99, device="cpu"):
    data, ep_returns = [], []
    for _ in range(G):
        env = env_fn()
        s, _ = env.reset()
        ep = {"s":[], "a":[], "logp":[], "r":[], "done":[], "s_next":[]}
        for t in range(max_steps):
            s_t = torch.tensor(s, dtype=torch.float32, device=device)
            d = policy.dist(s_t)
            a = d.sample()
            logp = d.log_prob(a).item()
            s2, r, terminated, truncated, _ = env.step(a.item())
            ep["s"].append(s); ep["a"].append(a.item()); ep["logp"].append(logp)
            ep["r"].append(r); ep["done"].append(terminated or truncated); ep["s_next"].append(s2)
            s = s2
            if terminated or truncated: break
        # Monte-Carlo return
        R, Gt = 0.0, []
        for r in reversed(ep["r"]):
            R = r + gamma * R
            Gt.append(R)
        ep["G"] = list(reversed(Gt))
        ep_returns.append(sum(ep["r"]))
        data.append(ep)
        env.close()
    return data, np.array(ep_returns, dtype=np.float32)

def build_batch(data, advantages):
    S,A,LOGP,ADV = [],[],[],[]
    for ep, adv in zip(data, advantages):
        n = len(ep["s"])
        S.append(torch.tensor(ep["s"], dtype=torch.float32))
        A.append(torch.tensor(ep["a"], dtype=torch.int64))
        LOGP.append(torch.tensor(ep["logp"], dtype=torch.float32))
        ADV.append(torch.full((n,), adv, dtype=torch.float32))   # outcome supervision
    return torch.cat(S), torch.cat(A), torch.cat(LOGP), torch.cat(ADV)

def estimate_group_advantages_outcome(ep_returns, eps=1e-8):
    m, s = ep_returns.mean(), ep_returns.std()
    if s < eps: s = eps
    return (ep_returns - m) / s

def kl_to_ref(policy, policy_ref, states):
    with torch.no_grad():
        p_ref = policy_ref.dist(states)
    p = policy.dist(states)
    # D_KL(p || p_ref)
    return torch.distributions.kl_divergence(p, p_ref).mean()


In [2]:
# ---- training loop ----
env_id = "LunarLander-v3"
def make_env(): return gym.make(env_id)

obs_dim = make_env().observation_space.shape[0]
act_dim = make_env().action_space.n

device = "cuda" if torch.cuda.is_available() else "cpu"
pi = Policy(obs_dim, act_dim).to(device)
pi_ref = Policy(obs_dim, act_dim).to(device)
pi_ref.load_state_dict(pi.state_dict())  # initial reference

opt = optim.Adam(pi.parameters(), lr=3e-4)

G = 16; T = 1200; epochs = 8; minibatches = 16
clip_eps = 0.2; beta_kl = 0.02; ent_coef = 0.01

for it in range(200):
    data, epR = rollout_group(make_env, pi, G, max_steps=T, device=device)
    adv_ep = estimate_group_advantages_outcome(epR)           # outcome supervision
    # (optional) normalize again:
    adv_ep = (adv_ep - adv_ep.mean()) / (adv_ep.std()+1e-8)

    S, A, LOGP_OLD, ADV = build_batch(data, adv_ep)
    perm = torch.randperm(len(S))
    S, A, LOGP_OLD, ADV = S[perm].to(device), A[perm].to(device), LOGP_OLD[perm].to(device), ADV[perm].to(device)

    for _ in range(epochs):
        for mb in torch.chunk(torch.arange(len(S)), minibatches):
            s, a, logp_old, adv = S[mb], A[mb], LOGP_OLD[mb], ADV[mb]
            dist = pi.dist(s)
            logp = dist.log_prob(a)
            ratio = (logp - logp_old).exp()
            unclipped = ratio * adv
            clipped = torch.clamp(ratio, 1.0-clip_eps, 1.0+clip_eps) * adv
            L_clip = torch.min(unclipped, clipped).mean()
            ent = dist.entropy().mean()
            dkl = kl_to_ref(pi, pi_ref, s)
            loss = -(L_clip - beta_kl*dkl + ent_coef*ent)
            opt.zero_grad(); loss.backward(); nn.utils.clip_grad_norm_(pi.parameters(), 0.5); opt.step()

    # Update reference occasionally (or keep fixed)
    if (it+1) % 20 == 0:
        pi_ref.load_state_dict(pi.state_dict())
    print(f"Iter {it} | avg return {epR.mean():.1f} | KL {dkl.item():.4f}")


/home/nvta/miniconda3/envs/rel301m/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/tmp/ipykernel_42904/3981165671.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  S.append(torch.tensor(ep["s"], dtype=torch.float32))


Iter 0 | avg return -194.1 | KL 0.0181
Iter 1 | avg return -172.5 | KL 0.0183
Iter 2 | avg return -196.9 | KL 0.0390
Iter 3 | avg return -132.1 | KL 0.0366
Iter 4 | avg return -98.9 | KL 0.0402
Iter 5 | avg return -148.8 | KL 0.0589
Iter 6 | avg return -123.4 | KL 0.0658
Iter 7 | avg return -139.1 | KL 0.0675
Iter 8 | avg return -155.0 | KL 0.0735
Iter 9 | avg return -134.0 | KL 0.0452
Iter 10 | avg return -137.6 | KL 0.0609
Iter 11 | avg return -134.8 | KL 0.0556
Iter 12 | avg return -124.7 | KL 0.0407
Iter 13 | avg return -138.0 | KL 0.0623
Iter 14 | avg return -111.2 | KL 0.0443
Iter 15 | avg return -112.1 | KL 0.0284
Iter 16 | avg return -130.2 | KL 0.0398
Iter 17 | avg return -115.6 | KL 0.0592
Iter 18 | avg return -130.8 | KL 0.0355


KeyboardInterrupt: 

In [35]:
import gymnasium as gym
import torch, torch.nn as nn, torch.optim as optim 
import numpy as np
from torch.distributions.categorical import Categorical
from multiprocessing import Pool, get_context
from functools import partial
from pathlib import Path
import time
import os, time
import multiprocessing as mp
torch.set_num_threads(1)  # avoid OpenMP storms inside each worker

# === Policy ===
class Policy(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, act_dim)
        )
    def forward(self, x): return self.net(x)
    def dist(self, x): return Categorical(logits=self(x))

# === Parallel rollout function ===
# ---- replace rollout_group_parallel with CPU workers and fork ----
def rollout_one(env_id, state_dict_cpu, max_steps=256, gamma=0.99):
    """Run one rollout entirely on CPU (safe with fork)."""
    env = gym.make(env_id)
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.n
    policy = Policy(obs_dim, act_dim)               # CPU
    policy.load_state_dict(state_dict_cpu)
    policy.eval()

    s, _ = env.reset()
    ep = {"s":[], "a":[], "logp":[], "r":[], "done":[], "s_next":[]}
    for _ in range(max_steps):
        s_t = torch.tensor(s, dtype=torch.float32).unsqueeze(0)  # CPU tensor
        with torch.no_grad():
            dist = policy.dist(s_t)
            a = dist.sample()
            logp = dist.log_prob(a).item()
        s2, r, terminated, truncated, _ = env.step(a.item())
        ep["s"].append(s); ep["a"].append(a.item()); ep["logp"].append(logp)
        ep["r"].append(r); ep["done"].append(terminated or truncated); ep["s_next"].append(s2)
        s = s2
        if terminated or truncated:
            break

    # MC return
    R, Gt = 0.0, []
    for r in reversed(ep["r"]):
        R = r + gamma * R
        Gt.append(R)
    ep["G"] = list(reversed(Gt))
    env.close()
    return ep, sum(ep["r"])

def rollout_group_parallel(env_id, policy_gpu, G, max_steps=256, gamma=0.99, n_workers=None):
    """Fork-safe parallel rollouts on CPU; training stays on GPU."""
    # ship a CPU copy of weights (don’t pass CUDA tensors!)
    state_dict_cpu = {k: v.detach().cpu() for k, v in policy_gpu.state_dict().items()}
    if n_workers is None:
        n_workers = min(G, os.cpu_count() or 1)

    # Use 'fork' on Linux to avoid pickling issues
    if mp.get_start_method(allow_none=True) != "fork":
        try:
            mp.set_start_method("fork", force=True)
        except RuntimeError:
            pass  # already set (ok)

    with mp.Pool(processes=n_workers) as pool:
        results = pool.starmap(
            rollout_one,
            [(env_id, state_dict_cpu, max_steps, gamma) for _ in range(G)]
        )
    data, ep_returns = zip(*results)
    return list(data), np.array(ep_returns, dtype=np.float32)


# === Helper functions ===
def build_batch(data, advantages):
    S,A,LOGP,ADV = [],[],[],[]
    for ep, adv in zip(data, advantages):
        n = len(ep["s"])
        S.append(torch.tensor(ep["s"], dtype=torch.float32))
        A.append(torch.tensor(ep["a"], dtype=torch.int64))
        LOGP.append(torch.tensor(ep["logp"], dtype=torch.float32))
        ADV.append(torch.full((n,), adv, dtype=torch.float32))
    return torch.cat(S), torch.cat(A), torch.cat(LOGP), torch.cat(ADV)

def estimate_group_advantages_outcome(ep_returns, eps=1e-8):
    m, s = ep_returns.mean(), ep_returns.std()
    if s < eps: s = eps
    return (ep_returns - m) / s

def kl_to_ref(policy, policy_ref, states):
    with torch.no_grad():
        p_ref = policy_ref.dist(states)
    p = policy.dist(states)
    return torch.distributions.kl_divergence(p, p_ref).mean()

# === Training loop ===
env_id = "LunarLander-v3"
env = gym.make(env_id)
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.n
env.close()

device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
pi = Policy(obs_dim, act_dim).to(device)
pi_ref = Policy(obs_dim, act_dim).to(device)
pi_ref.load_state_dict(pi.state_dict())

opt = optim.Adam(pi.parameters(), lr=3e-4)
G = 32             # one rollout per CPU core
T = 1024
epochs = 8
minibatches = 16
clip_eps = 0.2
beta_kl = 0.05
ent_coef = 0.01

for it in range(300):
    t0 = time.time()
    data, epR = rollout_group_parallel(env_id, pi, G, max_steps=T, n_workers=24)
    adv_ep = estimate_group_advantages_outcome(epR)
    adv_ep = (adv_ep - adv_ep.mean()) / (adv_ep.std() + 1e-8)

    S, A, LOGP_OLD, ADV = build_batch(data, adv_ep)
    perm = torch.randperm(len(S))
    S, A, LOGP_OLD, ADV = S[perm].to(device), A[perm].to(device), LOGP_OLD[perm].to(device), ADV[perm].to(device)

    for _ in range(epochs):
        for mb in torch.chunk(torch.arange(len(S)), minibatches):
            s, a, logp_old, adv = S[mb], A[mb], LOGP_OLD[mb], ADV[mb]
            dist = pi.dist(s)
            logp = dist.log_prob(a)
            ratio = (logp - logp_old).exp()
            unclipped = ratio * adv
            clipped = torch.clamp(ratio, 1.0-clip_eps, 1.0+clip_eps) * adv
            L_clip = torch.min(unclipped, clipped).mean()
            ent = dist.entropy().mean()
            dkl = kl_to_ref(pi, pi_ref, s)
            loss = -(L_clip - beta_kl*dkl + ent_coef*ent)
            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(pi.parameters(), 0.5)
            opt.step()

    if (it+1) % 20 == 0:
        pi_ref.load_state_dict(pi.state_dict())

    dt = time.time() - t0
    print(f"Iter {it} | avgR {epR.mean():.1f} | KL {dkl.item():.4f} | time {dt:.2f}s")


Iter 0 | avgR -235.4 | KL 0.0116 | time 0.99s
Iter 1 | avgR -188.0 | KL 0.0155 | time 1.33s
Iter 2 | avgR -104.3 | KL 0.0252 | time 1.34s
Iter 3 | avgR -116.3 | KL 0.0282 | time 1.01s
Iter 4 | avgR -128.7 | KL 0.0300 | time 0.97s
Iter 5 | avgR -166.1 | KL 0.0370 | time 0.96s
Iter 6 | avgR -121.3 | KL 0.0255 | time 1.05s
Iter 7 | avgR -143.9 | KL 0.0521 | time 1.24s
Iter 8 | avgR -191.6 | KL 0.0620 | time 1.26s
Iter 9 | avgR -119.6 | KL 0.0524 | time 1.33s
Iter 10 | avgR -151.3 | KL 0.0554 | time 1.04s
Iter 11 | avgR -112.0 | KL 0.0457 | time 1.09s
Iter 12 | avgR -121.5 | KL 0.0572 | time 1.07s
Iter 13 | avgR -161.4 | KL 0.1003 | time 1.03s
Iter 14 | avgR -111.2 | KL 0.0541 | time 1.04s
Iter 15 | avgR -111.9 | KL 0.0565 | time 1.03s
Iter 16 | avgR -103.2 | KL 0.0837 | time 0.97s
Iter 17 | avgR -119.0 | KL 0.0924 | time 1.30s
Iter 18 | avgR -108.0 | KL 0.0739 | time 1.13s
Iter 19 | avgR -110.6 | KL 0.0650 | time 0.99s
Iter 20 | avgR -115.6 | KL 0.0067 | time 1.04s
Iter 21 | avgR -134.7 |

In [7]:
# ===== PPO for LunarLander-v3 (parallel, apples-to-apples with your GRPO) =====
import gymnasium as gym
import torch, torch.nn as nn, torch.optim as optim
import numpy as np, os, time, multiprocessing as mp
from torch.distributions.categorical import Categorical

torch.set_num_threads(1)

# === Actor-Critic ===
class ActorCritic(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.pi = nn.Sequential(
            nn.Linear(obs_dim, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, act_dim)
        )
        self.v = nn.Sequential(
            nn.Linear(obs_dim, 128), nn.Tanh(),
            nn.Linear(128, 128), nn.Tanh(),
            nn.Linear(128, 1)
        )
    def dist(self, x): return Categorical(logits=self.pi(x))
    def value(self, x): return self.v(x).squeeze(-1)

# === Worker rollout ===
def rollout_one_ppo(env_id, state_dict_cpu, max_steps=256, gamma=0.99):
    env = gym.make(env_id)
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.n

    ac = ActorCritic(obs_dim, act_dim)  # CPU
    ac.load_state_dict(state_dict_cpu)
    ac.eval()

    s, _ = env.reset()
    ep = {"s":[], "a":[], "logp":[], "r":[], "done":[], "v":[], "s_next":[]}

    for _ in range(max_steps):
        s_t = torch.tensor(s, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            d = ac.dist(s_t)
            a = d.sample()
            logp = d.log_prob(a).item()
            v = ac.value(s_t).item()

        s2, r, terminated, truncated, _ = env.step(a.item())
        ep["s"].append(s); ep["a"].append(a.item()); ep["logp"].append(logp)
        ep["r"].append(r); ep["done"].append(terminated or truncated)
        ep["v"].append(v); ep["s_next"].append(s2)
        s = s2
        if terminated or truncated: break

    # bootstrap value for final state
    with torch.no_grad():
        s_last = torch.tensor(s, dtype=torch.float32).unsqueeze(0)
        v_last = ac.value(s_last).item()

    env.close()
    return ep, sum(ep["r"]), v_last

def rollout_group_parallel_ppo(env_id, ac_gpu, G, max_steps=256, gamma=0.99, n_workers=None):
    # ship CPU weights safely (no CUDA tensors)
    state_dict_cpu = {k: v.detach().cpu() for k, v in ac_gpu.state_dict().items()}
    if n_workers is None:
        n_workers = min(G, os.cpu_count() or 1)
    if mp.get_start_method(allow_none=True) != "fork":
        try: mp.set_start_method("fork", force=True)
        except RuntimeError: pass

    with mp.Pool(processes=n_workers) as pool:
        results = pool.starmap(
            rollout_one_ppo,
            [(env_id, state_dict_cpu, max_steps, gamma) for _ in range(G)]
        )

    data, ep_returns, v_last = zip(*results)
    return list(data), np.array(ep_returns, dtype=np.float32), np.array(v_last, dtype=np.float32)

# === GAE advantages ===
def compute_gae_for_episode(r, v, v_last, done, gamma=0.99, lam=0.95):
    # r, v are lists; done is list of bools (episode end flags)
    T = len(r)
    adv = np.zeros(T, dtype=np.float32)
    lastgaelam = 0.0
    for t in reversed(range(T)):
        nextv = v_last if t == T-1 else v[t+1]
        nonterminal = 0.0 if done[t] else 1.0
        delta = r[t] + gamma * nextv * nonterminal - v[t]
        lastgaelam = delta + gamma * lam * nonterminal * lastgaelam
        adv[t] = lastgaelam
    ret = adv + np.array(v, dtype=np.float32)
    return adv, ret

# === Build flat batch ===
def build_minibatch(data, v_last_list, gamma=0.99, lam=0.95):
    S, A, LOGP_OLD, ADV, RET = [], [], [], [], []
    for ep, v_last in zip(data, v_last_list):
        adv, ret = compute_gae_for_episode(ep["r"], ep["v"], v_last, ep["done"], gamma, lam)
        S.append(torch.tensor(ep["s"], dtype=torch.float32))
        A.append(torch.tensor(ep["a"], dtype=torch.int64))
        LOGP_OLD.append(torch.tensor(ep["logp"], dtype=torch.float32))
        ADV.append(torch.tensor(adv, dtype=torch.float32))
        RET.append(torch.tensor(ret, dtype=torch.float32))
    S = torch.cat(S); A = torch.cat(A); LOGP_OLD = torch.cat(LOGP_OLD)
    ADV = torch.cat(ADV); RET = torch.cat(RET)
    # normalize advantages
    ADV = (ADV - ADV.mean()) / (ADV.std() + 1e-8)
    return S, A, LOGP_OLD, ADV, RET

# === Train ===
def train_ppo(
    env_id="LunarLander-v3",
    total_iters=50,
    G=16, T=1024,
    epochs=16, minibatches=16,
    gamma=0.99, lam=0.95,
    clip_eps=0.2, vf_coef=0.5, ent_coef=0.01,
    lr=3e-4, max_grad_norm=0.5,
    device=None
):
    env = gym.make(env_id)
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.n
    env.close()

    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    # print(device)
    device = 'cpu'
    ac = ActorCritic(obs_dim, act_dim).to(device)
    opt = optim.Adam(ac.parameters(), lr=lr)

    for it in range(total_iters):
        t0 = time.time()
        data, epR, v_last = rollout_group_parallel_ppo(env_id, ac, G, max_steps=T, gamma=gamma)
        S, A, LOGP_OLD, ADV, RET = build_minibatch(data, v_last, gamma=gamma, lam=lam)

        # shuffle & move
        idx = torch.randperm(len(S))
        S, A, LOGP_OLD, ADV, RET = S[idx].to(device), A[idx].to(device), LOGP_OLD[idx].to(device), ADV[idx].to(device), RET[idx].to(device)

        for _ in range(epochs):
            for mb in torch.chunk(torch.arange(len(S)), minibatches):
                s, a, logp_old, adv, ret = S[mb], A[mb], LOGP_OLD[mb], ADV[mb], RET[mb]
                dist = ac.dist(s)
                logp = dist.log_prob(a)
                ratio = (logp - logp_old).exp()

                # Policy clip loss
                L_clip = torch.min(ratio * adv, torch.clamp(ratio, 1.0 - clip_eps, 1.0 + clip_eps) * adv).mean()

                # Value loss (MSE)
                v_pred = ac.value(s)
                v_loss = 0.5 * (ret - v_pred).pow(2).mean()  # unclipped; can add value clipping if desired

                # Entropy bonus
                ent = dist.entropy().mean()

                loss = -(L_clip) + vf_coef * v_loss - ent_coef * ent
                opt.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(ac.parameters(), max_grad_norm)
                opt.step()

        dt = time.time() - t0
        print(f"[PPO] Iter {it} | avgR {epR.mean():.1f} | time {dt:.2f}s")

    return ac


In [8]:
from dense_scripts.utils import record_videos
model = train_ppo()
device = "cuda"
env_id = "LunarLander-v3"
model.to(device)
record_videos(model, env_id, video_dir=f"videos/PPO", episodes=6, device=device)

[PPO] Iter 0 | avgR -142.8 | time 1.18s
[PPO] Iter 1 | avgR -292.3 | time 1.17s
[PPO] Iter 2 | avgR -295.6 | time 1.12s
[PPO] Iter 3 | avgR -345.6 | time 1.02s
[PPO] Iter 4 | avgR -263.3 | time 1.04s
[PPO] Iter 5 | avgR -191.1 | time 1.13s
[PPO] Iter 6 | avgR -125.2 | time 1.11s
[PPO] Iter 7 | avgR -205.2 | time 1.15s
[PPO] Iter 8 | avgR -224.4 | time 1.17s
[PPO] Iter 9 | avgR -105.3 | time 1.15s
[PPO] Iter 10 | avgR -108.7 | time 1.14s
[PPO] Iter 11 | avgR -114.3 | time 1.11s
[PPO] Iter 12 | avgR -174.8 | time 1.09s
[PPO] Iter 13 | avgR -141.5 | time 1.20s
[PPO] Iter 14 | avgR -198.9 | time 1.07s
[PPO] Iter 15 | avgR -189.2 | time 1.17s
[PPO] Iter 16 | avgR -82.2 | time 1.31s
[PPO] Iter 17 | avgR -90.9 | time 1.20s
[PPO] Iter 18 | avgR -116.9 | time 1.42s
[PPO] Iter 19 | avgR -104.6 | time 1.23s
[PPO] Iter 20 | avgR -22.2 | time 1.59s
[PPO] Iter 21 | avgR -119.1 | time 1.29s
[PPO] Iter 22 | avgR -54.8 | time 1.19s
[PPO] Iter 23 | avgR -56.7 | time 1.16s
[PPO] Iter 24 | avgR -64.1 | ti

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/LunarLander-v3_ep01_R235.3.mp4 | Reward: 235.3


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/LunarLander-v3_ep02_R-6.5.mp4 | Reward: -6.5


...

✅ Saved MP4 video: videos/PPO/LunarLander-v3_ep04_R253.9.mp4 | Reward: 253.9


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/LunarLander-v3_ep05_R237.5.mp4 | Reward: 237.5


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/LunarLander-v3_ep06_R232.9.mp4 | Reward: 232.9


In [8]:
import gymnasium as gym
from pathlib import Path
import imageio

# === After training ===
def record_video(policy, env_id="LunarLander-v3", video_dir="videos", episodes=3, device="cpu"):
    """
    Record video episodes of the trained GRPO agent.
    """
    env = gym.make(env_id, render_mode="rgb_array")
    Path(video_dir).mkdir(exist_ok=True)

    for ep in range(episodes):
        frames = []
        s, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            s_t = torch.tensor(s, dtype=torch.float32, device=device)
            with torch.no_grad():
                dist = policy.dist(s_t)
                a = dist.probs.argmax().item()   # Greedy action for visualization
            s, r, terminated, truncated, _ = env.step(a)
            done = terminated or truncated
            total_reward += r
            frames.append(env.render())

        # Save the video
        video_path = Path(video_dir) / f"episode_{ep+1:02d}_R{total_reward:.1f}.mp4"
        imageio.mimsave(video_path, frames, fps=30)
        print(f"Saved: {video_path} | Total reward: {total_reward:.1f}")

    env.close()


In [ ]:
from gymnasium.envs.box2d.lunar_lander import LunarLander
import numpy as np


class SparseLunarLander(LunarLander):
    """
    Sparse reward version of LunarLander by directly inheriting from LunarLander.
    
    The original LunarLander provides dense rewards at each step based on:
    - Distance to landing pad
    - Velocity
    - Angle
    - Leg contact
    - Fuel usage
    
    This sparse version only provides rewards at episode termination:
    - +100 for successful landing (both legs on ground, stable)
    - Penalty for crashes, scaled by:
      * Crash velocity (softer crashes get less penalty)
      * Distance from landing pad (farther = worse penalty)
      * Number of legs touching (landing on legs is better than body crash)
    """
    
    def __init__(self, success_reward=100.0, max_crash_penalty=-120.0, 
                 velocity_penalty_scale=50.0, leg_bonus_scale=20.0,
                 distance_penalty_scale=15.0, tilt_penalty_scale=15.0, 
                 out_of_bounds_penalty=-5.0, max_episode_steps=1000, **kwargs):
        """
        Initialize the sparse reward LunarLander.
        
        Args:
            success_reward: Reward for successful landing (default: 100.0)
            max_crash_penalty: Maximum penalty for worst crash (default: -100.0)
            velocity_penalty_scale: Scale for velocity-based penalty (default: 30.0)
            leg_bonus_scale: Bonus reduction for each leg touching (default: 20.0)
            distance_penalty_scale: Scale for distance-from-pad penalty (default: 10.0)
            tilt_penalty_scale: Scale for tilt/angle penalty (default: 15.0)
            out_of_bounds_penalty: Additional penalty for flying off screen (default: -40.0)
            max_episode_steps: Maximum number of steps per episode (default: 1000)
            **kwargs: Additional arguments passed to LunarLander constructor
        """
        super().__init__(**kwargs)
        
        self.success_reward = success_reward
        self.max_crash_penalty = max_crash_penalty
        self.velocity_penalty_scale = velocity_penalty_scale
        self.leg_bonus_scale = leg_bonus_scale
        self.distance_penalty_scale = distance_penalty_scale
        self.tilt_penalty_scale = tilt_penalty_scale
        self.out_of_bounds_penalty = out_of_bounds_penalty
        self.max_episode_steps = max_episode_steps
        
        # Track episode statistics
        self.episode_steps = 0
        self.total_original_reward = 0.0
        
    def reset(self, **kwargs):
        """Reset the environment and tracking variables."""
        self.episode_steps = 0
        self.total_original_reward = 0.0
        return super().reset(**kwargs)
    
    def step(self, action):
        """
        Override step method to provide sparse rewards.
        
        Args:
            action: The action to take
            
        Returns:
            observation, sparse_reward, terminated, truncated, info
        """
        # Call parent's step method to get original behavior
        observation, original_reward, terminated, truncated, info = super().step(action)
        
        # Track original reward for comparison
        self.total_original_reward += original_reward
        self.episode_steps += 1
        
        # Check if we've exceeded the step limit
        if self.episode_steps >= self.max_episode_steps:
            truncated = True
            info['TimeLimit.truncated'] = True
        
        # Calculate sparse reward - no reward during episode, only at termination
        sparse_reward = 0.0
        
        if terminated or truncated:
            # Check if this was a successful landing or a crash
            if original_reward > 0:
                # Successful landing (lander came to rest - not awake)
                sparse_reward += self.success_reward
                info['landing_success'] = True
                info['crash_velocity'] = 0.0
                info['legs_touching'] = 2
                info['distance_from_pad'] = 0.0
                info['out_of_bounds'] = False
            else:
                # Failed landing - calculate penalty based on crash severity
                info['landing_success'] = False
                
                # Get position (observation[0] is x position, landing pad is at x=0)
                x_position = observation[0]
                y_position = observation[1]
                distance_from_pad = np.sqrt(x_position**2 + y_position**2)
                
                # Check if lander flew off screen
                # LunarLander terminates when abs(state[0]) >= 1.0 (see lunar_lander.py line 658)
                # The observation space bounds are [-2.5, 2.5] but termination happens at [-1.0, 1.0]
                # Y position doesn't trigger termination, only x position does
                out_of_bounds = abs(x_position) >= 1.0
                info['out_of_bounds'] = out_of_bounds
                
                # Get velocity at crash (from observation)
                # observation[2] = horizontal velocity, observation[3] = vertical velocity
                vx = observation[2]
                vy = observation[3]
                crash_velocity = np.sqrt(vx**2 + vy**2)
                
                # Get angle/tilt at crash (observation[4] is angle in radians)
                # Angle of 0 means upright, larger angles mean more tilted
                angle = abs(observation[4])
                
                # Check how many legs were touching
                legs_touching = int(self.legs[0].ground_contact) + int(self.legs[1].ground_contact)
                
                # Calculate velocity-based penalty (continuous)
                # Higher velocity = worse penalty
                velocity_penalty = -self.velocity_penalty_scale * crash_velocity
                
                # Calculate distance-based penalty (continuous)
                # Farther from landing pad = worse penalty
                # Landing pad is at x=0, typical crashes are within [-1, 1] range
                distance_penalty = -self.distance_penalty_scale * distance_from_pad
                
                # Calculate tilt-based penalty (continuous)
                # More tilted = worse penalty
                tilt_penalty = -self.tilt_penalty_scale * angle
                
                # Reduce penalty for each leg touching (bonus for landing on legs)
                leg_bonus = self.leg_bonus_scale * legs_touching
                
                # Add extra penalty if flew off screen
                oob_penalty = self.out_of_bounds_penalty if out_of_bounds else 0.0
                
                # Total crash penalty (clamped to max_crash_penalty)
                crash_penalty = velocity_penalty + distance_penalty + tilt_penalty + leg_bonus + oob_penalty
                crash_penalty = max(crash_penalty, self.max_crash_penalty)
                
                sparse_reward += crash_penalty
                
                # Store crash details in info
                info['crash_velocity'] = float(crash_velocity)
                info['crash_angle'] = float(angle)
                info['distance_from_pad'] = float(distance_from_pad)
                info['legs_touching'] = legs_touching
                info['velocity_penalty'] = float(velocity_penalty)
                info['distance_penalty'] = float(distance_penalty)
                info['tilt_penalty'] = float(tilt_penalty)
                info['leg_bonus'] = float(leg_bonus)
                info['out_of_bounds_penalty'] = float(oob_penalty)
                info['crash_penalty'] = float(crash_penalty)
            
            # Add statistics to info
            info['sparse_reward'] = sparse_reward
            info['original_total_reward'] = self.total_original_reward
            info['episode_length'] = self.episode_steps
        
        return observation, sparse_reward, terminated, truncated, info

In [6]:
from dense_scripts.GRPO.grpo import PerStepAdvGRPO, GRPOConfig
from dense_scripts.utils.policies import SimpleGRPOPolicy

gamma = 0.98
cfg = GRPOConfig(
    env_id="LunarLander-v3",
    G=32, T=1024, epochs=8, minibatches=16, gamma=gamma,
    ent_coef=0.01, beta_kl=0.02, target_kl=0.015,
    n_workers=24, log_dir="./runs/GRPO_Lander"
)

pi = SimpleGRPOPolicy(obs_dim=8, act_dim=4, hidden=128)

trainer = PerStepAdvGRPO(policy=pi, config=cfg, device="cpu")
pi_star = trainer.train(iters=300)


GRPO:   0%|▏                                                        | 1/300 [00:01<05:14,  1.05s/it, KL=0.009, avgR=-147.6, beta=0.0133, it_s=1.05]

Iter 0000 | avgR -147.59 ± 80.85 | KL 0.0086 (β=0.01333) | Lclip 0.0073 Lkl 0.0086 Lent 1.3729 | steps 2925 eps 32 | time 1.05s


GRPO:   1%|▍                                                        | 2/300 [00:02<05:02,  1.01s/it, KL=0.016, avgR=-160.7, beta=0.0133, it_s=0.98]

Iter 0001 | avgR -160.71 ± 93.40 | KL 0.0162 (β=0.01333) | Lclip 0.0162 Lkl 0.0162 Lent 1.3381 | steps 6407 eps 64 | time 0.98s


GRPO:   1%|▌                                                       | 3/300 [00:03<04:57,  1.00s/it, KL=0.007, avgR=-128.3, beta=0.00889, it_s=0.98]

...

Iter 0297 | avgR  143.59 ± 68.99 | KL 0.0030 (β=0.0001) | Lclip 0.0017 Lkl 0.0030 Lent 0.9266 | steps 7101244 eps 9536 | time 2.01s


GRPO: 100%|███████████████████████████████████████████████████████▊| 299/300 [10:23<00:02,  2.10s/it, KL=0.004, avgR=140.3, beta=0.0001, it_s=1.92]

Iter 0298 | avgR  140.32 ± 69.37 | KL 0.0036 (β=0.0001) | Lclip 0.0007 Lkl 0.0036 Lent 0.9437 | steps 7124222 eps 9568 | time 1.92s


GRPO: 100%|████████████████████████████████████████████████████████| 300/300 [10:26<00:00,  2.09s/it, KL=0.004, avgR=131.1, beta=0.0001, it_s=2.47]

Iter 0299 | avgR  131.13 ± 53.75 | KL 0.0039 (β=0.0001) | Lclip 0.0014 Lkl 0.0039 Lent 0.9404 | steps 7152469 eps 9600 | time 2.47s


In [7]:
from dense_scripts.utils import record_videos

device = "cuda"
env_id = "LunarLander-v3"
pi_star.to(device)
record_videos(pi_star, env_id, video_dir=f"videos/GRPO/gamma_{gamma}", episodes=6, device=device)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98/LunarLander-v3_ep01_R231.5.mp4 | Reward: 231.5


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98/LunarLander-v3_ep02_R235.5.mp4 | Reward: 235.5


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


...

✅ Saved MP4 video: videos/GRPO/gamma_0.98/LunarLander-v3_ep04_R258.4.mp4 | Reward: 258.4


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98/LunarLander-v3_ep05_R128.2.mp4 | Reward: 128.2


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98/LunarLander-v3_ep06_R227.7.mp4 | Reward: 227.7


In [3]:
from dense_scripts.GRPO.grpo import PerStepAdvGRPO, GRPOConfig
from dense_scripts.utils.policies import SimpleGRPOPolicy
from dense_scripts.utils import record_videos

gamma = 0.98
G=32
env_id = "LunarLander-v3"
cfg = GRPOConfig(
    env_id="LunarLander-v3",
    G=G, T=1024, epochs=8, minibatches=32, gamma=gamma,
    ent_coef=0.01, beta_kl=0.02, target_kl=0.015,
    n_workers=24, log_dir="./runs/GRPO_Lander"
)

pi = SimpleGRPOPolicy(obs_dim=8, act_dim=4, hidden=128)

trainer = PerStepAdvGRPO(policy=pi, config=cfg, device="cpu")
pi_star = trainer.train(iters=300)

device = "cuda"
pi_star.to(device)
record_videos(pi_star, env_id, video_dir=f"videos/GRPO/gamma_{gamma}_{G}", episodes=12, device=device)

GRPO:   0%|▏                                                        | 1/300 [00:01<06:32,  1.31s/it, KL=0.010, avgR=-132.2, beta=0.0133, it_s=1.31]

Iter 0000 | avgR -132.17 ± 76.77 | KL 0.0098 (β=0.01333) | Lclip 0.0071 Lkl 0.0098 Lent 1.3735 | steps 2838 eps 32 | time 1.31s


GRPO:   1%|▎                                                       | 2/300 [00:02<06:24,  1.29s/it, KL=0.007, avgR=-109.5, beta=0.00889, it_s=1.27]

Iter 0001 | avgR -109.48 ± 35.21 | KL 0.0072 (β=0.008889) | Lclip 0.0061 Lkl 0.0072 Lent 1.3506 | steps 5592 eps 64 | time 1.27s


GRPO:   1%|▌                                                       | 3/300 [00:03<06:07,  1.24s/it, KL=0.010, avgR=-107.2, beta=0.00889, it_s=1.17]

...

✅ Saved MP4 video: videos/GRPO/gamma_0.98_32/LunarLander-v3_ep10_R243.9.mp4 | Reward: 243.9


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98_32/LunarLander-v3_ep11_R134.0.mp4 | Reward: 134.0


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.98_32/LunarLander-v3_ep12_R97.9.mp4 | Reward: 97.9


In [11]:
record_videos(pi_star, env_id, video_dir=f"videos/GRPO/gamma_{gamma}", episodes=6, device=device)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975/LunarLander-v3_ep01_R209.8.mp4 | Reward: 209.8


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975/LunarLander-v3_ep02_R193.2.mp4 | Reward: 193.2


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


...

✅ Saved MP4 video: videos/GRPO/gamma_0.975/LunarLander-v3_ep04_R210.8.mp4 | Reward: 210.8


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975/LunarLander-v3_ep05_R169.3.mp4 | Reward: 169.3


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975/LunarLander-v3_ep06_R217.5.mp4 | Reward: 217.5


In [5]:
from dense_scripts.utils import record_videos
device = "cuda"
env_id = "LunarLander-v3"
pi_star.to(device)
record_videos(pi_star, env_id, video_dir=f"videos/GRPO/gamma_{gamma}_{G}", episodes=12, device=device)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975_64/LunarLander-v3_ep01_R195.2.mp4 | Reward: 195.2


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975_64/LunarLander-v3_ep02_R159.3.mp4 | Reward: 159.3


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


...

✅ Saved MP4 video: videos/GRPO/gamma_0.975_64/LunarLander-v3_ep10_R149.5.mp4 | Reward: 149.5


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975_64/LunarLander-v3_ep11_R221.1.mp4 | Reward: 221.1


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/GRPO/gamma_0.975_64/LunarLander-v3_ep12_R167.5.mp4 | Reward: 167.5


In [ ]:
from dense_scripts.GRPO.grpo import PerStepAdvGRPOTrainer, GRPOConfig
from dense_scripts.utils.policies import SimpleGRPOPolicy

gamma = 0.98
G = 32
cfg = GRPOConfig(
    env_id="LunarLander-v3",
    G=G, T=1024, epochs=8, minibatches=32, gamma=gamma,
    ent_coef=0.01, beta_kl=0.02, target_kl=0.015,
    n_workers=24, log_dir="./runs/GRPO"
)

pi = SimpleGRPOPolicy(obs_dim=8, act_dim=4, hidden=128)
trainer = PerStepAdvGRPOTrainer(policy=pi, config=cfg, device="cpu")

# automatically logs + saves config + records video at the end
pi_star = trainer.train(
    iters=30,
    video_dir="videos/GRPO",
    video_fps=30,
    video_format="mp4",
    video_episodes=2
)


/home/nvta/miniconda3/envs/rel301m/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/home/nvta/miniconda3/envs/rel301m/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📝 Saved hyperparameters to runs/GRPO/grpo_01h23_08112025/grpo_01h23_08112025.txt


GRPO (01h23_08112025):   0%|                                                                                                | 0/30 [00:00<?, ?it/s]/home/nvta/NVTA/REL-Project/dense_scripts/GRPO/grpo.py:125: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  S.append(torch.tensor(ep["s"], dtype=torch.float32))
GRPO (01h23_08112025):   3%|█▍                                         | 1/30 [00:01<00:37,  1.30s/it, KL=0.011, avgR=-137.2, beta=0.02, it_s=1.30]

Iter 0000 | avgR -137.21 ± 64.06 | KL 0.0110 (β=0.02) | Lclip 0.0076 Lkl 0.0110 Lent 1.3626 | steps 3053 eps 32 | time 1.30s


GRPO (01h23_08112025):   7%|██▊                                        | 2/30 [00:02<00:32,  1.15s/it, KL=0.014, avgR=-255.0, beta=0.02, it_s=1.05]

Iter 0001 | avgR -255.04 ±111.17 | KL 0.0137 (β=0.02) | Lclip 0.0140 Lkl 0.0137 Lent 1.3447 | steps 6025 eps 64 | time 1.05s


In [6]:
from dense_scripts.PPO import PPOTrainer, PPOConfig
from dense_scripts.utils.policies import SimpleActorCriticPolicy

cfg = PPOConfig(
    env_id="LunarLander-v3",
    G=24, T=1024,
    epochs=8, minibatches=32,
    gamma=0.99, lam=0.95,
    lr=3e-4,
    vf_coef=0.5, ent_coef=0.01,
    objective="clip",        # or "klpen"
    target_kl=0.015,
    beta_kl=0.9,
    n_workers=24,
    log_dir="./runs/PPO"
)

pi = SimpleActorCriticPolicy(obs_dim=8, act_dim=4)

trainer = PPOTrainer(policy=pi, config=cfg, device="cpu")

pi_star, v_star = trainer.train(
    iters=50,
    video_dir="videos/PPO",
    video_fps=30,
    video_format="mp4",
    video_episodes=6
)

PPO (03h04_08112025):   2%|█▏                                                       | 1/50 [00:02<02:18,  2.82s/it, KL=0.011, avgR=-178, it_s=2.82]

Iter 000 | avgR -178.5 ±116.1 | KL 0.0105 β=0.900 (clip) | Lπ -0.0223 Lv 798.36 H 1.377 | steps 24576 | 2.82s


PPO (03h04_08112025):   4%|██▎                                                      | 2/50 [00:05<02:17,  2.87s/it, KL=0.020, avgR=-158, it_s=2.90]

Iter 001 | avgR -158.5 ±110.1 | KL 0.0203 β=0.900 (clip) | Lπ -0.0241 Lv 615.02 H 1.371 | steps 49152 | 2.90s


PPO (03h04_08112025):   6%|███▍                                                     | 3/50 [00:07<01:55,  2.45s/it, KL=0.030, avgR=-120, it_s=1.94]

...

✅ Saved MP4 video: videos/PPO/G24_γ0.99_03h04_08112025/LunarLander-v3_ep04_R-69.3.mp4 | Reward: -69.3


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/G24_γ0.99_03h04_08112025/LunarLander-v3_ep05_R-61.0.mp4 | Reward: -61.0


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


✅ Saved MP4 video: videos/PPO/G24_γ0.99_03h04_08112025/LunarLander-v3_ep06_R-28.6.mp4 | Reward: -28.6
